In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from statsmodels.api import OLS
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("../input/big-mart-sales-prediction/Train.csv")
test = pd.read_csv("../input/big-mart-sales-prediction/Test.csv")
test1 = test.copy()
print('Train data shape:', train.shape)
print('Test data shape:',test.shape)

In [ ]:
train.head()

In [ ]:
test["Outlet_Size"].unique()

In [ ]:
train.nunique()

In [ ]:
test.nunique()

In [ ]:
train.isnull().sum()

In [ ]:
map1 = {"Small":1,"Medium":2,"High":3}
train["Outlet_Size"] = train["Outlet_Size"].map(map1)
train["Item_Weight"] = train["Item_Weight"].fillna(train.Item_Weight.mean())
train["Outlet_Size"] = train["Outlet_Size"].fillna(train["Outlet_Size"].median())

In [ ]:
train.isnull().sum()

In [ ]:
map1 = {"Small":1,"Medium":2,"High":3}
test["Outlet_Size"] = test["Outlet_Size"].map(map1)
test["Item_Weight"] = test["Item_Weight"].fillna(test.Item_Weight.mean())
test["Outlet_Size"] = test["Outlet_Size"].fillna(test["Outlet_Size"].median())

In [ ]:
train.head()

In [ ]:
train['Item_Outlet_Sales'].plot(legend=True,label='SALES',figsize=(12,8))

In [ ]:
f, ax = plt.subplots(1,1, figsize=(10, 6))
plot = sns.lineplot(y='Item_Outlet_Sales', x='Outlet_Establishment_Year', data=train)
plot.set(title='Sales Data')

In [ ]:
plt.figure(figsize=(15,6))
plt.scatter(y='Outlet_Establishment_Year', x='Outlet_Identifier', data=train, marker = '*')
plt.suptitle('Establishment of Outlet in Year')

**Checking Data Distribution for Item_MRP and Item_Outlet_Sales**

In [ ]:
plt.hist(train["Item_Outlet_Sales"],bins = 100)


In [ ]:
plt.hist(train["Item_MRP"],alpha = 0.3,bins = 150, color='cyan')

In [ ]:
fig,axes=plt.subplots(1,1,figsize=(12,8))
sns.scatterplot(x='Item_MRP',y='Item_Outlet_Sales',hue='Item_Fat_Content',size='Item_Weight',data=train)

In [ ]:
plt.figure(figsize=(7,4))
sns.countplot(data= train, x= "Outlet_Type")

In [ ]:
sns.countplot(train["Outlet_Location_Type"])
plt.show()

In [ ]:
plt.figure(figsize=(25,8))
sns.countplot(data= train, x= "Item_Type")

In [ ]:
sns.countplot(train["Outlet_Size"])
plt.show()

In [ ]:
sns.countplot(train["Outlet_Type"],palette = 'RdYlGn')
plt.xticks(rotation = 90)
plt.show()

In [ ]:
sns.violinplot(x=train["Outlet_Size"],y=train["Item_Outlet_Sales"],hue = train["Outlet_Size"],palette = "RdYlGn")
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize= (9,6))
corr= train.corr()
sns.heatmap(corr, linewidths=1.5, annot= True, cmap='RdYlGn')

In [ ]:
train.drop(labels = ["Outlet_Establishment_Year"],inplace = True,axis =1)
test.drop(labels = ["Outlet_Establishment_Year"],inplace = True,axis =1)

In [ ]:
feat = ['Outlet_Size','Outlet_Type','Outlet_Location_Type','Item_Fat_Content',"Item_Type"]
X = pd.get_dummies(train[feat])
train = pd.concat([train,X],axis=1)

In [ ]:
train.head()

In [ ]:
feat = ['Outlet_Size','Outlet_Type','Outlet_Location_Type','Item_Fat_Content',"Item_Type"]
X1 = pd.get_dummies(test[feat])
test = pd.concat([test,X1],axis=1)

In [ ]:
train.drop(labels = ["Outlet_Size",'Outlet_Location_Type',"Outlet_Type",'Item_Fat_Content','Outlet_Identifier','Item_Identifier',"Item_Type"],axis=1,inplace = True)
test.drop(labels = ["Outlet_Size",'Outlet_Location_Type',"Outlet_Type",'Item_Fat_Content','Outlet_Identifier','Item_Identifier',"Item_Type"],axis=1,inplace = True)

In [ ]:
X_train = train.drop(labels = ["Item_Outlet_Sales"],axis=1)
y_train = train["Item_Outlet_Sales"]
X_train.shape,y_train.shape

In [ ]:
train.head()

In [ ]:
y_train.head()

In [ ]:
from sklearn import preprocessing

x = X_train.values #returns a numpy array
test_s = test.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled_train = min_max_scaler.fit_transform(x)
x_scaled_test = min_max_scaler.fit_transform(test_s)
df_train = pd.DataFrame(x_scaled_train)
df_test = pd.DataFrame(x_scaled_test)

In [ ]:
df_train.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train, y_train, test_size=0.4)


In [ ]:
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledElastic', Pipeline([('Scaler', StandardScaler()),('EN',ElasticNet())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledDT', Pipeline([('Scaler', StandardScaler()),('DT', DecisionTreeRegressor())])))
pipelines.append(('ScaledRF', Pipeline([('Scaler', StandardScaler()),('RF', RandomForestRegressor())])))
pipelines.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GB', GradientBoostingRegressor())])))
pipelines.append(('ScaledAda', Pipeline([('Scaler', StandardScaler()),('Ada', AdaBoostRegressor())])))
pipelines.append(('ScaledETR', Pipeline([('Scaler', StandardScaler()),('ETR', ExtraTreesRegressor())])))
pipelines.append(('ScaledSVR', Pipeline([('Scaler', StandardScaler()),('SVM', SVR())])))
pipelines.append(('ScaledXGB', Pipeline([('Scaler', StandardScaler()),('XGBR', XGBRegressor())])))
pipelines.append(('ScaledMLP', Pipeline([('Scaler', StandardScaler()),('NNW', MLPRegressor())])))

In [ ]:
results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=21)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_root_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Algorithm comparison
fig = plt.figure(figsize=(18,5))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

# **Linear Regression**

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)

In [ ]:
lr_preds = model.predict(X_test)


In [ ]:
plt.scatter(y_test, lr_preds)
plt.show()

In [ ]:
lr_predictions = model.predict(df_test)
final = pd.DataFrame({"Item_Identifier":test1["Item_Identifier"],"Outlet_Identifier":test1["Outlet_Identifier"],"Item_Outlet_Sales":abs(lr_predictions)})
final.head()

In [ ]:
sns.distplot((y_test-lr_preds),bins=50)
plt.show()


In [ ]:
from sklearn import metrics
print('RMSE of Linear Regression:', np.sqrt(metrics.mean_squared_error(y_test, lr_preds)))

# **Gradient Boosting Regressor**

In [ ]:
gbm= GradientBoostingRegressor().fit(X_train, y_train)
gbm_preds= gbm.predict(X_test)
print('RMSE of GB:', np.sqrt(mean_squared_error(gbm_preds, y_test)))

# **XGBoost Regressor**

In [ ]:
xgb_model= XGBRegressor().fit(X_train, y_train)
xgb_preds= xgb_model.predict(X_test)
print('RMSE of XGBoost:', np.sqrt(mean_squared_error(xgb_preds, y_test)))

Out of all models Gradient Boosting Regressor is performing well in terms of RMSE. Hence finalizing it as final model for prediction.

In [ ]:
plt.scatter(y_test, gbm_preds)
plt.show()

In [ ]:
sns.distplot((y_test-gbm_preds),bins=50)
plt.show()

In [ ]:
gb_predictions = gbm.predict(df_test)
final = pd.DataFrame({"Item_Identifier":test1["Item_Identifier"],"Outlet_Identifier":test1["Outlet_Identifier"],"Item_Outlet_Sales":abs(gb_predictions)})
final.head()

In [ ]:
final.to_csv('/kaggle/working/Submission_GBoostRegressor.csv', index=False)